In [95]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import pandas as pd

In [12]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '183aea69820e4dfda0176db816fc1f72' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

In [16]:
# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage2 } 
data = None
params = None

In [14]:
# Face detect funciton

def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

In [17]:
# Image path
urlImage2 = 'https://raw.githubusercontent.com/Microsoft/ProjectOxford-ClientSDK/master/Face/Windows/Data/detection3.jpg'

In [87]:
# Running functions
result = processRequest( json, data, headers, params,  )
face = detect(urlImage2)

In [88]:
# Adding gender to emotions
final_data = []
for i, value in enumerate(result):
    final_data.append(result[i])
    final_data[i].update(face[i])

[{'faceRectangle': {'height': 140, 'left': 216, 'top': 143, 'width': 140},
  'scores': {'anger': 4.857832e-09,
   'contempt': 3.60886387e-09,
   'disgust': 1.88275351e-09,
   'fear': 8.208998e-10,
   'happiness': 0.99999994,
   'neutral': 1.58002855e-08,
   'sadness': 2.58834572e-08,
   'surprise': 3.569176e-08}},
 {'faceRectangle': {'height': 138, 'left': 378, 'top': 239, 'width': 138},
  'scores': {'anger': 4.67109421e-05,
   'contempt': 9.014235e-05,
   'disgust': 7.946587e-05,
   'fear': 0.0564681776,
   'happiness': 5.95064048e-05,
   'neutral': 0.5410432,
   'sadness': 0.299806863,
   'surprise': 0.102405921}}]

In [ ]:
#Turning Json to pandas dataframe